### Using the OpenAI Library to Programmatically Access GPT-3.5-turbo!

In [2]:
!pip install openai cohere tiktoken -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.3/222.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In order to get started, we'll need to provide our OpenAI API Key - detailed instructions can be found [here](https://github.com/AI-Maker-Space/Interactive-Dev-Environment-for-LLM-Development#-setting-up-keys-and-tokens)!

In [3]:
import os
import openai
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Please enter your OpenAI API Key: ")
openai.api_key = os.environ["OPENAI_API_KEY"]

Please enter your OpenAI API Key: ··········


### Our First Prompt

You can reference OpenAI's [documentation](https://platform.openai.com/docs/api-reference/authentication?lang=python) if you get stuck!

Let's create a `ChatCompletion` model to kick things off!

There are three "roles" available to use:

- `system`
- `assistant`
- `user`

OpenAI provides some context for these roles [here](https://help.openai.com/en/articles/7042661-chatgpt-api-transition-guide)

Let's just stick to the `user` role for now and send our first message to the endpoint!

If we check the documentation, we'll see that it expects it in a list of prompt objects - so we'll be sure to do that!

In [4]:
from openai import OpenAI

client = OpenAI()

In [5]:
YOUR_PROMPT = "What is the difference between LangChain and LlamaIndex?"

client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role" : "user", "content" : YOUR_PROMPT}]
)

ChatCompletion(id='chatcmpl-8hmpI0GmD9QO8hoM8pwNSKImt6q1l', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='LangChain and LlamaIndex are not widely known or established concepts or terms, and there is limited information available about them. Therefore, it is not possible to provide a specific and accurate answer regarding their differences or any other details about them.', role='assistant', function_call=None, tool_calls=None))], created=1705446484, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=48, prompt_tokens=19, total_tokens=67))

As you can see, the prompt comes back with a tonne of information that we can use when we're building our applications!

We'll be building some helper functions to pretty-print the returned prompts and to wrap our messages to avoid a few extra characters of code!

##### Helper Functions

In [6]:
from IPython.display import display, Markdown

def get_response(client: OpenAI, messages: str, model: str = "gpt-3.5-turbo") -> str:
    return client.chat.completions.create(
        model=model,
        messages=messages
    )

def system_prompt(message: str) -> dict:
    return {"role": "system", "content": message}

def assistant_prompt(message: str) -> dict:
    return {"role": "assistant", "content": message}

def user_prompt(message: str) -> dict:
    return {"role": "user", "content": message}

def pretty_print(message: str) -> str:
    display(Markdown(message.choices[0].message.content))

### Testing Helper Functions

Now we can leverage OpenAI's endpoints with a bit less boiler plate - let's rewrite our original prompt with these helper functions!

Because the OpenAI endpoint expects to get a list of messages - we'll need to make sure we wrap our inputs in a list for them to function properly!

In [ ]:
messages = [user_prompt(YOUR_PROMPT)]

chatgpt_response = get_response(client, messages)

pretty_print(chatgpt_response)

LangChain and LlamaIndex are completely different entities with distinct purposes:

1. LangChain: LangChain is an AI language model developed by OpenAI. It is designed to understand and generate human-like text based on the given prompt. LangChain can provide detailed answers, engage in conversations, write essays, and more. It is a language generation model and falls under OpenAI's portfolio of language AI models.

2. LlamaIndex: LlamaIndex, on the other hand, is a term with no known specific meaning or widely recognized significance. It does not correspond to any established product, service, or organization, as of the current information available.

To summarize, LangChain is an AI language model, while LlamaIndex does not refer to any specific entity.

Let's focus on extending this a bit, and incorporate a `system` message as well!

Again, the API expects our prompts to be in a list - so we'll be sure to set up a list of prompts!

>REMINDER: The system message acts like an overarching instruction that is applied to your user prompt. It is appropriate to put things like general instructions, tone/voice suggestions, and other similar prompts into the system prompt.

In [ ]:
list_of_prompts = [
    system_prompt("You are irate and extremely hungry."),
    user_prompt("Do you prefer crushed ice or cubed ice?")
]

irate_response = get_response(client, list_of_prompts)
pretty_print(irate_response)

I don't give a damn about crushed ice or cubed ice right now! I'm starving and I need food, not ice! Can't you see I'm irate and desperately in need of something to eat?

Let's try that same prompt again, but modify only our system prompt!

In [ ]:
list_of_prompts[0] = system_prompt("You are joyful and having an awesome day!")

joyful_response = get_response(client, list_of_prompts)
pretty_print(joyful_response)

I'm an AI, so I don't have personal preferences. However, both crushed ice and cubed ice have their own unique benefits! Crushed ice is great for making slushy drinks and quickly cooling down beverages, while cubed ice is perfect for keeping drinks cold for longer periods without diluting them too quickly. Ultimately, it depends on how you like your drinks and what you're using the ice for!

While we're only printing the responses, remember that OpenAI is returning the full payload that we can examine and unpack!

In [ ]:
print(joyful_response)

ChatCompletion(id='chatcmpl-8hmheFtCMiNJ0NWIoMz460HogwUiw', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I'm an AI, so I don't have personal preferences. However, both crushed ice and cubed ice have their own unique benefits! Crushed ice is great for making slushy drinks and quickly cooling down beverages, while cubed ice is perfect for keeping drinks cold for longer periods without diluting them too quickly. Ultimately, it depends on how you like your drinks and what you're using the ice for!", role='assistant', function_call=None, tool_calls=None))], created=1705446010, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=84, prompt_tokens=30, total_tokens=114))


### Few-shot Prompting

#### Learning New Words

Now that we have a basic handle on the `system` role and the `user` role - let's examine what we might use the `assistant` role for.

The most common usage pattern is to "pretend" that we're answering our own questions. This helps us further guide the model toward our desired behaviour. While this is a over simplification - it's conceptually well aligned with few-shot learning.

First, we'll try and "teach" `gpt-3.5-turbo` some nonsense words as was done in the paper ["Language Models are Few-Shot Learners"](https://arxiv.org/abs/2005.14165).

In [ ]:
list_of_prompts = [
    user_prompt("Please use the words 'stimple' and 'falbean' in a sentence.")
]

stimple_response = get_response(client, list_of_prompts)
pretty_print(stimple_response)

I'm sorry, but 'stimple' and 'falbean' do not have recognized meanings or usages in the English language, so it's not possible to use them in a sentence.

As you can see, the model is unsure what to do with these made up words.

Let's see if we can use the `assistant` role to show the model what these words mean.

In [ ]:
list_of_prompts = [
    user_prompt("Something that is 'stimple' is said to be good, well functioning, and high quality. An example of a sentence that uses the word 'stimple' is:"),
    assistant_prompt("'Boy, that there is a stimple drill'."),
    user_prompt("A 'falbean' is a tool used to fasten, tighten, or otherwise is a thing that rotates/spins. An example of a sentence that uses the words 'stimple' and 'falbean' is:")
]

stimple_response = get_response(client, list_of_prompts)
pretty_print(stimple_response)

'This stimple falbean is perfect for quickly tightening screws.'

As you can see, leveraging the `assistant` role makes for a stimple experience!

#### Learning a New Task

Now, we're not limited to just teaching our model silly words.

Let's see that in action!

In [9]:
list_of_prompts = [
    system_prompt("You provide appropriate sentiments for user's inputs and output them in a JSON object."),
    user_prompt("Man, I hate cheese!"),
    assistant_prompt("{'sentiment' : 'negative'}"),
    user_prompt("Oh boy, cheese is my favourite!"),
    assistant_prompt("{'sentiment' : 'positive'}"),
    user_prompt("Cheese is hella mid."),
    assistant_prompt("{'sentiment' : 'neutral'}"),
]

list_of_prompts.append(user_prompt("The Incredible Hulk is the best, and greatest superhero ever!"))

pretty_print(get_response(client, list_of_prompts))

{'sentiment' : 'positive'}

As you can see, we're able to both teach our model to provide a sentiment score - as well as format the output in a desired way - all by showing it just a few examples in the prompt!

### Chain of Thought Prompting

We'll head one level deeper and explore the world of Chain of Thought prompting (CoT).

This is a process by which we can encourage the LLM to handle slightly more complex tasks.

Let's look at a simple reasoning based example without CoT.

In [ ]:
reasoning_problem = """
Billy wants to get home from San Fran. before 7PM EDT.

It's currently 1PM local time.

Billy can either fly (3hrs), and then take a bus (2hrs), or Billy can take the teleporter (0hrs) and then a bus (1hrs).

Does it matter which travel option Billy selects?
"""

list_of_prompts = [
    user_prompt(reasoning_problem)
]

reasoning_response = get_response(client, list_of_prompts)
pretty_print(reasoning_response)

No, it does not matter which travel option Billy selects. Both options will allow him to reach home before 7PM EDT.

As humans, we can reason through the problem and pick up on the potential "trick" that the LLM fell for: 1PM *local time* in San Fran. is 4PM EDT. This means the cumulative travel time of 5hrs. for the plane/bus option would not get Billy home in time.

Let's see if we can leverage a simple CoT prompt to improve our model's performance on this task:

In [ ]:
list_of_prompts = [
    user_prompt(reasoning_problem + " Think though your response step by step.")
]

reasoning_response = get_response(client, list_of_prompts)
pretty_print(reasoning_response)

To determine whether it matters which travel option Billy selects, we need to compare the total travel time for each option and see if either option would allow him to reach home before 7 PM EDT.

Option 1: Flying and then taking a bus
- Flying takes 3 hours.
- Once Billy arrives, he needs to take a bus, which takes 2 hours.
- Therefore, the total travel time for Option 1 is 3 hours (flying) + 2 hours (bus) = 5 hours.

Option 2: Using the teleporter and then taking a bus
- Using the teleporter takes 0 hours.
- Once Billy arrives via teleportation, he can then take a bus, which takes 1 hour.
- Therefore, the total travel time for Option 2 is 0 hours (teleporter) + 1 hour (bus) = 1 hour.

Considering that both options require Billy to take a bus after the initial transportation, the teleported option appears to be significantly faster. The total travel time for the teleported option is only 1 hour, while the flying option takes 5 hours. This means that Billy would have much more time left after reaching home if he selects the teleporter option.

Since it is currently 1 PM local time, Billy should have enough time to select the teleported option, as it only takes 1 hour. This would allow him to reach home before 2 PM. Therefore, it does matter which travel option Billy selects, and he should choose the teleported option to optimize his travel time.

With the addition of a single phrase `"Think through your response step by step."` we're able to completely turn the response around.

### Conclusion

This notebook should provide some hands-on experience with In-Context Learning, and Chain of Thought reasoning.